In [101]:
import pandas as pd
from mxnet import gluon, autograd, init, nd
from mxnet.gluon import nn
import mxnet as mx
import random
import matplotlib.pyplot as plt

In [68]:
dfraw = pd.read_csv('../data/FinalData.csv')

dfgp = dfraw.groupby(['SerialNumber', 'Name', 'City'])
for i,k in dfgp:
    print(i)


dfc = dfraw.drop_duplicates()
assert len(dfc) == len(dfraw)
len(dfc)

('4cvxlkt', 'aishamanzil', 'Karachi')
('4ypxth4', 'Defence Housing Authority', 'Karachi')
('997gv7h', 'northkarachi', 'Karachi')
('a67pmak', 'landhi', 'Karachi')
('aj9xpaw', 'Korangi', 'Karachi')
('ajgclt7', 'Korangi (J-Area)', 'Karachi')
('awawxwx', 'Fida Hussain House, Bedian Road', 'Lahore')
('cpguxuc', 'saddar', 'Karachi')
('crp4y7m', 'Peshawar Node', 'Peshawar')
('g6gs9ls', 'Hyderi Market', 'Karachi')
('kuh6xuc', 'Lahore Cantt', 'Lahore')
('kxvyx76', 'Lahore (Upper Mall)', 'Lahore')
('lllm4gm', 'bahadurabad', 'Karachi')
('lwpu64p', 'orangi town', 'Karachi')
('m9cuxv7', 'd.h.a phase 5', 'Karachi')
('rjpamt6', 'karachi - industrial', 'Karachi')
('sarpstt', 'Al Farooq School', 'Bahawalpur')
('tj4gawm', 'shershah', 'Karachi')
('tlxa6x7', 'Liaquatabad', 'Karachi')
('twa67s6', 'mt khan road', 'Karachi')
('u7cwkla', 'Sector E11', 'Islamabad')
('yggpgpl', 'Civic Center, Korangi', 'Karachi')


151256

In [22]:
dfc.columns

Index(['SerialNumber', 'City', 'Name', 'FirstOnline', 'Datetime', 'PM2.5',
       'USAQI', 'CO2', 'Temperature', 'RelativeHumidity', 'Outdoor PM2.5',
       'OutdoorUSAQI', 'weekday', 'weekdayName', 'hour', 'month', 'year',
       'dayofmonth', 'tmpc', 'relh', 'drct', 'sped', 'vsby'],
      dtype='object')

In [23]:
drop_columns = ['City', 'Name', 'FirstOnline', 'Outdoor PM2.5', 'OutdoorUSAQI', 'weekdayName']
df = dfc.drop(drop_columns, axis = 1)

In [24]:
df.isna().mean() * 100

SerialNumber         9.891178
Datetime             0.000000
PM2.5                0.000000
USAQI                0.000000
CO2                  9.891178
Temperature          0.991035
RelativeHumidity     0.991035
weekday              0.000000
hour                 0.000000
month                0.000000
year                 0.000000
dayofmonth           0.000000
tmpc                22.101603
relh                22.483736
drct                22.043423
sped                21.995161
vsby                21.975327
dtype: float64

In [67]:
df= df.dropna()
len(df)

KeyError: 'Name'

In [26]:
df['Datetime'] = pd.to_datetime(df.Datetime)

In [32]:
dfs = df.sort_values(['SerialNumber', 'Datetime'])
dfs = dfs.reset_index(drop = True)

In [33]:
batch_size = 16

In [34]:
def make_dataset_many_to_one(dataframe, time_steps):
    rows = dataframe.shape[0] - time_steps
    cols = dataframe.shape[1]
    dfnew = nd.zeros(shape = (rows, cols * time_steps))
    for i in range(0, time_steps):
        dfnew[:, i * cols : (i + 1 ) * cols] = dataframe.iloc[i: i + rows,:]
    ynew = nd.zeros(shape = (rows, 1))
    ynew[:,0] = dataframe['PM2.5'].iloc[time_steps:]
    return dfnew, ynew

In [50]:
def myPartSerial(i, time_steps):
    dfslice = dfs[dfs.SerialNumber == i]
    lis = []
        
    dfdiff = dfslice.Datetime.diff()
    dfdiff.iloc[0] = dfdiff.iloc[1]
    dfdiff2 = dfdiff != pd.Timedelta('1 hour')
    dfdiff3 = dfdiff2.cumsum()    
    
    for j,k in dfslice.groupby(dfdiff3):
        if len(k) > time_steps:
            d = k.iloc[:]
            drop_columns2 = ['SerialNumber', 'Datetime', 'year', 'relh', 'tmpc']
            d = d.drop(drop_columns2, axis = 1)
            lis.append( d )
    
    newlis = []
    newlabels = []
    for i in lis:
        a, b = make_dataset_many_to_one(i, time_steps)
        assert a.shape[0] == b.shape[0]
        newlis.append(a)
        newlabels.append(b)
    
 
    newlis = nd.concatenate(newlis)
    newlabels = nd.concatenate(newlabels)
#    newlis = nd.array(newlis)
#    print(newlist.shape)
    
    
    return newlis, newlabels

In [51]:
dfs.head(5)

,SerialNumber,Datetime,PM2.5,USAQI,CO2,Temperature,RelativeHumidity,weekday,hour,month,year,dayofmonth,tmpc,relh,drct,sped,vsby
0,4cvxlkt,2018-01-03 00:00:00,50.0,137,471.0,27.0,64.0,3,0,3,2018,1,21.0,68.55,230.0,4.60,3.73
1,4cvxlkt,2018-01-03 01:00:00,44.0,122,483.0,27.0,64.0,3,1,3,2018,1,21.0,73.09,235.0,4.60,3.73
2,4cvxlkt,2018-01-03 02:00:00,48.0,132,422.0,27.0,61.0,3,2,3,2018,1,21.0,68.55,260.0,5.75,3.42
3,4cvxlkt,2018-01-03 03:00:00,44.0,122,408.0,26.0,61.0,3,3,3,2018,1,22.0,64.48,125.0,2.30,2.49
4,4cvxlkt,2018-01-03 04:00:00,48.0,132,420.0,26.0,62.0,3,4,3,2018,1,25.0,59.25,125.0,2.30,2.49


In [72]:
lis = []
labelslis = []
time_step = 4

for i in dfs.SerialNumber.unique()[:]:
    li, la = myPartSerial(i, time_step)
    lis.append(li)
    labelslis.append(la)
    
lis = nd.concatenate(lis)
labelslis = nd.concatenate(labelslis)

print(lis.shape)
print(labelslis.shape)

(90666, 48)
(90666, 1)


In [89]:
test_ratio = 0.3
col_count = lis.shape[1] / time_step
assert col_count == int(col_count)
col_count = int(col_count)

train_count = int((1 - test_ratio) * lis.shape[0])
train_count = train_count // col_count * col_count

train_x = lis[:train_count]
total_x = lis[:]

trn_x = train_x.reshape(-1, time_step, col_count)
tot_x = total_x.reshape(-1, time_step, col_count)
trn_y = labelslis[:train_count].reshape(-1)
tot_y = labelslis[:].reshape(-1)


In [135]:
class Net(gluon.Block):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            
            self.lstm = gluon.rnn.LSTM(120,1,dropout=0.2)
            self.out = gluon.nn.Dense(1)
            
    def forward(self,x):
        
        x = self.lstm(x)
        x = self.out(x)
        
        return x
    
net = Net()

In [136]:
train_iter = mx.io.NDArrayIter(trn_x, trn_y, batch_size, shuffle=True)
device = mx.gpu(0) if mx.context.num_gpus() > 0 else mx.cpu(0)

In [137]:
lr = 0.1
net.initialize(force_reinit=True,  init=init.Xavier(), ctx=device)
loss = gluon.loss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

In [138]:
def eva(net, X, y, l, plot = False):
    y_hat = net(X)
    if plot:
        for i in range(0, min(len(y), 150), 50):
            plt.plot(y[i : i+50].asnumpy())
            plt.plot(y_hat[i : i+50].asnumpy())
            plt.show()

    return l(y_hat, y).mean().asscalar()

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    for trn_batch in train_iter:

        X = trn_batch.data[0].as_in_context(device)
        y = trn_batch.label[0].as_in_context(device)
        
        
        
        with autograd.record():
            y_pred = net(X)
            l = loss(y_pred, y)
        
        l.backward()
        
        trainer.step(batch_size = batch_size)
    
    train_iter.reset()
    
    print('epoch number ', epoch)
    print("traingin data loss\t", eva(net, trn_x.as_in_context(device), trn_y, loss) )
    print("testing data loss\t", eva(net, tot_x.as_in_context(device), tot_y, loss) )
    